In [61]:
!jupyter nbconvert --to html "C:/xampp/htdocs/ml_personalNGroup/Group/challenge3/notebooks/02-feature-engineering.ipynb"

[NbConvertApp] Converting notebook C:/xampp/htdocs/ml_personalNGroup/Group/challenge3/notebooks/02-feature-engineering.ipynb to html
[NbConvertApp] Writing 313632 bytes to C:\xampp\htdocs\ml_personalNGroup\Group\challenge3\notebooks\02-feature-engineering.html


## Tải dữ liệu

In [44]:
df_train = pd.read_csv('../data/01_raw/train.csv')
df_test = pd.read_csv('../data/01_raw/test.csv')
test_ids = df_test['Id']

In [45]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3600 entries, 0 to 3599
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  3600 non-null   int64  
 1   Artist Name         3600 non-null   object 
 2   Track Name          3600 non-null   object 
 3   Popularity          3505 non-null   float64
 4   danceability        3600 non-null   float64
 5   energy              3600 non-null   float64
 6   key                 3195 non-null   float64
 7   loudness            3600 non-null   float64
 8   mode                3600 non-null   int64  
 9   speechiness         3600 non-null   float64
 10  acousticness        3600 non-null   float64
 11  instrumentalness    2764 non-null   float64
 12  liveness            3600 non-null   float64
 13  valence             3600 non-null   float64
 14  tempo               3600 non-null   float64
 15  duration_in min/ms  3600 non-null   float64
 16  time_s

Xử lý missing values
Sửa dữ liệu lỗi (duration, loudness)
Log transform các feature skew
One-hot encode các cột categorical
Clip outliers hợp lý
Normalize numerical features nếu dùng SVM/KNN
Xử lý imbalance bằng class-weight hoặc SMOTE
Drop hoặc encode Artist Name tùy chiến lược

## Xử lý Missing Values

In [46]:
df_train['Popularity'] = df_train['Popularity'].fillna(df_train['Popularity'].mean())
df_train['key'] = df_train['key'].fillna(df_train['key'].mode()[0])
df_train['instrumentalness'] = df_train['instrumentalness'].fillna(0)
df_test['key'] = df_test['key'].fillna(df_test['key'].mode()[0])
df_test['instrumentalness'] = df_test['instrumentalness'].fillna(0)
df_test['Popularity'] = df_test['Popularity'].fillna(df_train['Popularity'].mean())

In [47]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14396 entries, 0 to 14395
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  14396 non-null  int64  
 1   Artist Name         14396 non-null  object 
 2   Track Name          14396 non-null  object 
 3   Popularity          14396 non-null  float64
 4   danceability        14396 non-null  float64
 5   energy              14396 non-null  float64
 6   key                 14396 non-null  float64
 7   loudness            14396 non-null  float64
 8   mode                14396 non-null  int64  
 9   speechiness         14396 non-null  float64
 10  acousticness        14396 non-null  float64
 11  instrumentalness    14396 non-null  float64
 12  liveness            14396 non-null  float64
 13  valence             14396 non-null  float64
 14  tempo               14396 non-null  float64
 15  duration_in min/ms  14396 non-null  float64
 16  time

## Sửa lỗi dữ liệu

(a) Cột duration_in min/ms bị lẫn đơn vị

In [48]:
df_train['duration_in min/ms']
df_test['duration_in min/ms']

0       200000.0
1       283909.0
2       232533.0
3       222053.0
4       118439.0
          ...   
3595    267987.0
3596    198300.0
3597    339213.0
3598    182587.0
3599    187733.0
Name: duration_in min/ms, Length: 3600, dtype: float64

In [49]:
mask_min = df_train['duration_in min/ms'] < 30
df_train.loc[mask_min, 'duration_in min/ms'] *= 60*1000
df_train.rename(columns={'duration_in min/ms': 'duration_in_ms'}, inplace=True)

mask_min_test = df_test['duration_in min/ms'] < 30
df_test.loc[mask_min_test, 'duration_in min/ms'] *= 60*1000
df_test.rename(columns={'duration_in min/ms': 'duration_in_ms'}, inplace=True)

(b) Cột loudness có giá trị dương

In [50]:
df_train['loudness'] = df_train['loudness'].clip(upper=0)
df_test['loudness'] = df_test['loudness'].clip(upper=0)

Kiểm tra còn gtri nào lớn hơn 0 kh 

In [51]:
df_train[df_train['loudness']>0]
df_test[df_test['loudness']>0]

,Id,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in_ms,time_signature


## Biến đổi phân phối lệch

In [52]:
import numpy as np
skew_cols = ['speechiness', 'acousticness', 'liveness', 'instrumentalness']
for col in skew_cols:
    df_train[col] = np.log1p(df_train[col])
    df_test[col] = np.log1p(df_test[col])
    

## Encoding biến phân loại

In [53]:
df_train = pd.get_dummies(df_train, columns=['key', 'mode', 'time_signature'], drop_first=True)
df_train.drop(columns=['Track Name'], inplace=True) 
df_test = pd.get_dummies(df_test, columns=['key', 'mode', 'time_signature'], drop_first=True)
df_test.drop(columns=['Track Name'], inplace=True)

In [54]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14396 entries, 0 to 14395
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Id                14396 non-null  int64  
 1   Artist Name       14396 non-null  object 
 2   Popularity        14396 non-null  float64
 3   danceability      14396 non-null  float64
 4   energy            14396 non-null  float64
 5   loudness          14396 non-null  float64
 6   speechiness       14396 non-null  float64
 7   acousticness      14396 non-null  float64
 8   instrumentalness  14396 non-null  float64
 9   liveness          14396 non-null  float64
 10  valence           14396 non-null  float64
 11  tempo             14396 non-null  float64
 12  duration_in_ms    14396 non-null  float64
 13  Class             14396 non-null  int64  
 14  key_2.0           14396 non-null  bool   
 15  key_3.0           14396 non-null  bool   
 16  key_4.0           14396 non-null  bool  

## Xử lý Outliers

Instrumentalness outlier theo IQR = "outlier giả" → KHÔNG XOÁ

tempo có outlier thật nhưng ít → có thể clip

duration_ms sau chuẩn hóa nên clip 1%–99%

In [55]:
df_train['tempo'] = df_train['tempo'].clip(lower=df_train['tempo'].quantile(0.01),
                               upper=df_train['tempo'].quantile(0.99))

df_train['duration_in_ms'] = df_train['duration_in_ms'].clip(lower=df_train['duration_in_ms'].quantile(0.01),
                                           upper=df_train['duration_in_ms'].quantile(0.99))

df_test['tempo'] = df_test['tempo'].clip(lower=df_test['tempo'].quantile(0.01),
                                 upper=df_test['tempo'].quantile(0.99))
df_test['duration_in_ms'] = df_test['duration_in_ms'].clip(lower=df_test['duration_in_ms'].quantile(0.01),
                                         upper=df_test['duration_in_ms'].quantile(0.99))

In [56]:
df_train[['tempo', 'duration_in_ms']].describe()

,tempo,duration_in_ms
count,14396.000000,14396.000000
mean,122.690984,233627.485135
std,29.192225,72059.831562
min,70.915650,113670.250000
25%,99.799000,187333.000000
50%,120.060000,219800.500000
75%,141.988250,262455.000000
max,193.754100,523353.000000


## Feature Scaling (Chuẩn hóa dữ liệu)

In [57]:
from sklearn.preprocessing import StandardScaler

scale_cols = ['danceability','energy','loudness','speechiness','acousticness',
              'instrumentalness','liveness','valence','tempo','duration_in_ms','Popularity']

scaler = StandardScaler()
df_train[scale_cols] = scaler.fit_transform(df_train[scale_cols])
df_test[scale_cols] = scaler.transform(df_test[scale_cols])

In [58]:
df_train.head()

,Id,Artist Name,Popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,...,key_6.0,key_7.0,key_8.0,key_9.0,key_10.0,key_11.0,mode_1,time_signature_3,time_signature_4,time_signature_5
0,1,Marina Maximilian,-0.437113,-1.263390,-0.535777,0.308703,-0.519565,0.562430,-0.508555,-0.590149,...,False,False,False,True,False,False,False,False,True,False
1,2,The Black Keys,1.305483,1.098986,0.358445,0.580863,0.134101,-0.733841,-0.282518,-0.605319,...,False,False,False,False,False,True,True,False,True,False
2,3,Royal & the Serpent,0.000000,0.247081,0.600012,0.445522,-0.201475,-0.847979,1.921148,0.660580,...,False,True,False,False,False,False,True,False,True,False
3,4,Detroit Blues Band,-1.889277,-0.169809,-1.502046,-1.678758,-0.613149,1.995926,-0.404395,0.764374,...,False,True,False,False,False,False,True,True,False,False
4,5,Coast Contra,0.201839,0.132285,0.485585,0.691551,2.075815,-0.110772,-0.508555,0.169997,...,True,False,False,False,False,False,False,False,True,False


## Xử lý Imbalanced Class (Class 10 quá nhiều)

In [59]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(df_train['Class'])
weights = compute_class_weight(class_weight='balanced', classes=classes, y=df_train['Class'])
class_weights = dict(zip(classes, weights))

Lưu file csv đã xử lí

In [60]:
import os
os.makedirs("../data/02_processed", exist_ok=True)
df_train.to_csv("../data/02_processed/train_clean.csv", index=False)
df_test.to_csv("../data/02_processed/test_clean.csv", index=False)
print("Dữ liệu đã sẵn sàng cho mô hình (03_processed).")

Dữ liệu đã sẵn sàng cho mô hình (03_processed).
